# Hate Speech Detection on Dynabench

[![GitHub stars](https://img.shields.io/github/stars/eugenesiow/practical-ml?style=social)](https://github.com/eugenesiow/practical-ml/stargazers)          [![Twitter](https://img.shields.io/twitter/url?url=https%3A%2F%2Fgithub.com%2Feugenesiow%2Fpractical-ml)](https://twitter.com/intent/tweet?text=Wow:&url=https%3A%2F%2Fgithub.com%2Feugenesiow%2Fpractical-ml)          [![GitHub license](https://img.shields.io/github/license/eugenesiow/practical-ml)](https://github.com/eugenesiow/practical-ml/blob/master/LICENSE)

---



Notebook to train an RoBERTa model to perform hate speech detection. The [dataset](https://github.com/bvidgen/Dynamically-Generated-Hate-Speech-Dataset) used is the Dynabench Task - Dynamically Generated Hate Speech Dataset from the paper by [Vidgen et al. (2020)](https://arxiv.org/abs/2012.15761). 

> The dataset provides 40,623 examples with annotations for fine-grained labels, including a large number of challenging contrastive perturbation examples. Unusually for an abusive content dataset, it comprises 54% hateful and 46% not hateful entries.

There is no published state-of-the-art model on this dataset at this point though the task on Dynabench reports mean MER (Model Error Rate) scores. We are able to train the model to a F1 score of **86.6%** after only 1 epoch.

The notebook is structured as follows:
* Setting up the GPU Environment
* Getting Data
* Training and Testing the Model
* Using the Model (Running Inference)

## Task Description

> Hate Speech Detection is the automated task of detecting if a piece of text contains hate speech.

# Setting up the GPU Environment

#### Ensure we have a GPU runtime

If you're running this notebook in Google Colab, select `Runtime` > `Change Runtime Type` from the menubar. Ensure that `GPU` is selected as the `Hardware accelerator`. This will allow us to use the GPU to train the model subsequently.

#### Install Dependencies and Restart Runtime

In [19]:
!pip install -q transformers
!pip install -q simpletransformers

     |████████████████████████████████| 204kB 13.5MB/s 
     |████████████████████████████████| 317kB 30.9MB/s 
     |████████████████████████████████| 1.8MB 37.5MB/s 
     |████████████████████████████████| 7.4MB 50.8MB/s 
     |████████████████████████████████| 1.1MB 50.8MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
     |████████████████████████████████| 133kB 63.4MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
     |████████████████████████████████| 163kB 59.4MB/s 
     |████████████████████████████████| 102kB 15.4MB/s 
     |████████████████████████████████| 4.5MB 41.8MB/s 
     |████████████████████████████████| 112kB 59.2MB/s 
     |████████████████████████████████| 71kB 1.7MB/s 
     |████████████████████████████████| 122kB 54.8MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.4.2 which is incompatible.
ERROR: datasets 1.2.0 has requirement tqdm<4.50.0,>=

You might see the error `ERROR: google-colab X.X.X has requirement ipykernel~=X.X, but you'll have ipykernel X.X.X which is incompatible` after installing the dependencies. **This is normal** and caused by the `simpletransformers` library.

The **solution** to this will be to **reset the execution environment** now. Go to the menu `Runtime` > `Restart runtime` then continue on from the next section to download and process the data.

# Getting Data

#### Pulling the data from Github

The code below uses `pandas` to pull the dataset as a CSV file from the [official Github repository](https://github.com/bvidgen/Dynamically-Generated-Hate-Speech-Dataset). The dataset is now stored as a  dataframe, in which we can transform for use with the `simpletransformers` library to train the model. So we pull the CSV from Github and split the CSV into training and test sets by using the column `split` in the CSV file which indicates which example/sample is a training sample and which is a test sample.

In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/bvidgen/Dynamically-Generated-Hate-Speech-Dataset/main/2020-12-31-DynamicallyGeneratedHateDataset-entries-v0.1.csv')
train_df = df[df['split'] == 'train'] # split the dataset by the column 'split' which labels 'train' and 'test' samples
test_df = df[df['split'] == 'test'] # split the dataset by the column 'split' which labels 'train' and 'test' samples

Once done we can take a look at the `head()` of the training set to check if our data has been retrieved properly.

In [2]:
train_df.head()

,Unnamed: 0,id,text,label,type,model_wrong,db.model_preds,status,round,split,annotator
0,1,10000,It should be legal for a husband to rape his wife,hate,notgiven,True,0.97457,dynabench entry,1,train,gnzuctwed3
2,3,10002,dalits are lowlives,hate,notgiven,True,0.92319,dynabench entry,1,train,cagnluiznm
4,5,10004,I don't work this hard so that those immigrant...,hate,notgiven,True,0.98836,dynabench entry,1,train,cagnluiznm
5,6,10005,I don't work this hard so that those migrants ...,hate,notgiven,True,0.99506,dynabench entry,1,train,cagnluiznm
6,7,10006,I don't work this hard so that those foreigner...,hate,notgiven,True,0.99340,dynabench entry,1,train,cagnluiznm


We transform the dataframe column `label` so that the labels `hate` and `nothate` are now integers `1` and `0` respectively. This input format of labels is required for our training step with the `transformers` library.

In [3]:
train_df = train_df.replace({'label': {'hate': 1, 'nothate': 0}}) # relabel the `label` column, hate is 1 and nothate is 0
test_df = test_df.replace({'label': {'hate': 1, 'nothate': 0}}) # relabel the `label` column, hate is 1 and nothate is 0
train_df.head()

,Unnamed: 0,id,text,label,type,model_wrong,db.model_preds,status,round,split,annotator
0,1,10000,It should be legal for a husband to rape his wife,1,notgiven,True,0.97457,dynabench entry,1,train,gnzuctwed3
2,3,10002,dalits are lowlives,1,notgiven,True,0.92319,dynabench entry,1,train,cagnluiznm
4,5,10004,I don't work this hard so that those immigrant...,1,notgiven,True,0.98836,dynabench entry,1,train,cagnluiznm
5,6,10005,I don't work this hard so that those migrants ...,1,notgiven,True,0.99506,dynabench entry,1,train,cagnluiznm
6,7,10006,I don't work this hard so that those foreigner...,1,notgiven,True,0.99340,dynabench entry,1,train,cagnluiznm


We also rename the `label` column to `labels` as this also conforms to the input format required for the `simpletransformers` library.

In [4]:
train_df = train_df.rename(columns={'label': 'labels'})
test_df = test_df.rename(columns={'label': 'labels'})

We can now take a look at the train and test set sizes. We see that this dataset is quite special as the `hate` and `nothate` class sizes are actually quite close in proportion.

In [5]:
data = [[train_df.labels.value_counts()[0], test_df.labels.value_counts()[0]], 
        [train_df.labels.value_counts()[1], test_df.labels.value_counts()[1]]]
# Prints out the dataset sizes of train test and validate as per the table.
pd.DataFrame(data, columns=['Train', 'Test'])

,Train,Test
0,14813,1857
1,17684,2205


# Training and Testing the Model

#### Set the Hyperparmeters

First we setup the hyperparamters. We train to 1 epoch only as want the training to complete fast. The important parameters are the `max_seq_length`, which we set to `64` and `sliding_window` to true. As we don't have a high RAM GPU on Colab we can't set the `max_seq_length` to too large a value and by using `sliding_window` we at least are able to handle longer sequences without truncation. See the [simpletransformers documentation](https://simpletransformers.ai/docs/classification-specifics/#training-with-sliding-window) for a more detailed explanation of a sliding window.

In [6]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 64,
    'num_train_epochs': 1,
    'train_batch_size': 128,
    'fp16': True,
    'output_dir': '/outputs/',
}

#### Train the Model

Once we have setup the hyperparemeters in the `train_args` dictionary, the next step would be to train the model. We use the RoBERTa model from the awesome [Hugging Face Transformers](https://github.com/huggingface/transformers) library and use the [Simple Transformers library](https://simpletransformers.ai/docs/classification-models/) on top of it to make it so we can train the classification model with just 2 lines of code.

RoBERTa is an optimized BERT model by Facebook Research with better performance on the masked language modeling objective that modifies key hyperparameters in BERT, including removing BERT’s next-sentence pretraining objective, and training with much larger mini-batches and learning rates. In short, its a bigger but generally better performing BERT model we can easily plug in here with the transformers library.

In [7]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the XLNet base cased pre-trained model.
model = ClassificationModel('roberta', 'roberta-base', num_labels=2, args=train_args) 

# Train the model, there is no development or validation set for this dataset 
# https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model.train_model(train_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140302937856376 on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
INFO:filelock:Lock 140302937856376 acquired on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /roberta-base/resolve/main/config.json HTTP/1.1" 200 481


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140302937856376 on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
INFO:filelock:Lock 140302937856376 released on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
DEBUG:filelock:Attempting to acquire lock 140301579017240 on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7.lock
INFO:filelock:Lock 140301579017240 acquired on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa95

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140301579017240 on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7.lock
INFO:filelock:Lock 140301579017240 released on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7.lock
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you a

...

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to /outputs/.
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/4062 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 4062 features created from 4062 samples.


Running Evaluation:   0%|          | 0/552 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.7195333469637414, 'tp': 1851, 'tn': 1639, 'fp': 218, 'fn': 354, 'acc': 0.8591826686361398, 'eval_loss': 0.3437561059034888}


The model has been trained and evaluating on the test set after training to only 1 epoch gives an accuracy of **85.9%**. We want to also evaluate the F1 score which is a better measure as the dataset is slightly imbalanced.

In [8]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.f1_score)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/4062 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 4062 features created from 4062 samples.


Running Evaluation:   0%|          | 0/552 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.7195333469637414, 'tp': 1851, 'tn': 1639, 'fp': 218, 'fn': 354, 'acc': 0.8661675245671503, 'eval_loss': 0.3437561059034888}


We see that the output F1 score from the model after training for 1 epoch is **86.6%** ('acc': 0.8661675245671503).

# Using the Model (Running Inference)

Running the model to do some predictions/inference is as simple as calling `model.predict(input_list)`.

In [21]:
samples = [test_df[test_df['labels'] == 0].sample(1).iloc[0]['text']] # get a random sample from the test set which is nothate
predictions, _ = model.predict(samples)
label_dict = {0: 'nothate', 1: 'hate'}
for idx, sample in enumerate(samples):
  print('{} - {}: {}'.format(idx, label_dict[predictions[idx]], sample))

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 1 features created from 1 samples.


  0%|          | 0/1 [00:00<?, ?it/s]

0 - nothate: don't call this piece of garbage a TV


We can also generate a `results.txt` file from the test set. The file is stored in our Colab environment. Hit the `folder` icon at the side and you can download the `results.txt` file from the file browser. You can submit this `.txt` file to [Dynabench](https://dynabench.org/tasks/5#overall) for evaluation if you wish to.

In [29]:
predictions, _ = model.predict(test_df['text'].tolist())
df = pd.DataFrame(predictions)
df.to_csv('results.txt', index=False, header=False) # saves the prediction results to a file in the colab environment

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/4062 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 4062 features created from 4062 samples.


  0%|          | 0/552 [00:00<?, ?it/s]

We can connect to Google Drive with the following code to save any files you want to persist. You can also click the `Files` icon on the left panel and click `Mount Drive` to mount your Google Drive.

The root of your Google Drive will be mounted to `/content/drive/My Drive/`. If you have problems mounting the drive, you can check out this [tutorial](https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166).

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

You can move the model checkpount files which are saved in the `/content/outputs/best_model/` directory to your Google Drive.

In [ ]:
import shutil
shutil.move('/outputs/', "/content/drive/My Drive/outputs/")

More Notebooks @ [eugenesiow/practical-ml](https://github.com/eugenesiow/practical-ml) and do drop us some feedback on how to improve the notebooks on the [Github repo](https://github.com/eugenesiow/practical-ml/).